# NO epsman local VM testing
18/03/21

Revisiting with new ePS version (build 3885d87, VM on Jake).

- NOTE in VM, ~/ePS > ~/ePS on Jake via VirtualBox folder share, hence output files will be directly on Jake (and not use VHD space).
- ePSman restructure version in use, but still in dev phase.
- Manual forcing of ePS bin path to desired new version.


Test runs + debug for NO on VM(Jake version):

- Got build running OK after remembering to set paths (need all libs etc.), now set in /opt/ePolyScat.3885d87/ePS_v3885d87_env.sh, which includes Intel config scripts.
- Job script ~/ePS/jobs/ePS-3885d87_batch_job.sh, need to run as source ./ePS-3885d87_batch_job.sh
- Hacked job writer shell script for new commands abSym and molden - NEED TO SORT THIS OUT AS IT'S SHIT.
- NOW seems to be running,
   - Outputs to .inp.err file for some reason, not sure what has changed.
   - `AbSym` + `molden` seem to work OK.
   - Now testing for all symmetries to see if there is a working set(s).
   - ALSO: orb groups & symmetries now in C2v, so had to change those too! Now finding some valid sets.
- Get 3 valid symmetry sets... but all drop out at ScatStab on intial test. Parameters or code or lib issue? (See cells below for more details). Results: `completed-NO-C2v-v2-ScatIssues`
   - This was with only occ orbs loaded, degen orb may be the issue?
   - Retry with CnvOrbSel=1,9... same results.
   - Any other params to play with here...?
- 23/03/21 NOW RUNNING!!!! Issue was spins, had set 2,3,4, but should be 2,3,2 (for init, final, total).
   - Running separate notebooks for E scans...
   - [O(1s)](http://localhost:6899/lab/tree/ePS/NO_2021/epsman_test_notebooks/NO_orb1_epsman_Jake_UB20VM-ePSv3885d87_230321.ipynb)


- Try `make testall` and check results.
   - Running for ub-ifort build with env settings.
   - `make testall` runs OK for this build.
   - Results in `/opt/ePolyScat.3885d87/tests/outdir.ub-ifort/`.
   - Checked a couple, look OK (completed). 
   - Test script not working, seem to be missing CompDiff.exe? Try a manual diff?

Test script ([as per manual](https://epolyscat.droppages.com/)):

```
. extras/CompDiff.sh # this defines a few shell functions
CompDiffStndAll tests/outdir.$MACH tests # compares the .out files in the first directory given with the .ostnd files in the second directory indicated
```




25/02/21

Revisiting with more symmetry + spin testing.

See method dev in http://localhost:8889/lab/tree/Scratch/ePS_jobs_Stimpy_2021/Xe_ePS_ePSman_dev_symTests_010221.ipynb


14/12/20

Trying to reup on NO ePS calcs + running locally on Win machine with UB20 VM. Previously connected OK remotely, so should be fine...?

Also: get back into epsman dev, and finish off some things!

# Set-up job

## Host configuration

In [45]:
!hostname

jake


In [2]:
import sys
from pathlib import Path

# Load dev scripts
# modPath = r'D:\code\github'
# sys.path.append(modPath)
import epsman as em
# import repo

In [3]:
# # Set machine, and local or remote run.
# machine = 'AntonJr'
# remoteFlag = True

# # Run machines.py to configure
# machScript = modPath + 'epsman/machines.py'
# %run -i {machScript}

# Configure with Class - set IP here or at prompt later.
# job = em.epsJob(host = 'AntonJr', user = 'paul', IP = '10.8.0.6')
# job = em.epsJob(host = 'UB20VM', user = 'paul', IP = '10.8.0.6')
job = em.epsJob(host = 'UB20VM', user = 'paul', IP = '192.168.86.82')  # For locally running VM, this works (with default ports), but 'localhost' does not.
# For more notes on this: https://serverfault.com/questions/1029251/is-it-possible-to-ssh-to-vm-via-localhost-in-bridged-mode-in-virtualbox
# https://medium.com/platform-engineer/port-forwarding-for-ssh-http-on-virtualbox-459277a888be
# https://nsrc.org/workshops/2014/btnog/raw-attachment/wiki/Track2Agenda/ex-virtualbox-portforward-ssh.htm

Set host = UB20VM
Set user = paul
Set IP = 192.168.86.82
Set password = None
Set mol = None
Set orb = None
Set batch = None
Set elecStructure = None
Set genFile = None
Set jobSettings = None
Skipping setJobPaths() until job settings defined, run setJob() to set.


In [4]:
# Check machine settings (currently in hard-coded dictionary)
# Will need to adjust for case when localhost is also ePS machine.
# job.hostDefn[job.host]

In [5]:
# ... and localhost
job.hostDefn['localhost']

{'host': 'jake',
 'IP': '127.0.0.1',
 'home': PosixPath('/home/paul'),
 'wrkdir': PosixPath('/home/paul/ePS/NO_2021/epsman_test_notebooks'),
 'webDir': PosixPath('/home/paul/github/ePSdata')}

In [6]:
# Setup and test connection + remote paths
job.initConnection()

Connecting to machine: UB20VM at 192.168.86.82


Password for machine?  ·····


Testing connection...
paul-VirtualBox
Connected OK
Command exited with status 0.
=== stdout ===
paul-VirtualBox

(no stderr)


Setting host dir tree.
Set remote wrkdir: /home/paul/ePS

***Default paths set
{'IP': '192.168.86.82',
 'condaEnv': 'base',
 'condaPath': PosixPath('/home/paul/anaconda3/bin/activate'),
 'ePSpath': PosixPath('/opt/ePolyScat.E3/bin/ePolyScat'),
 'home': PosixPath('/home/paul'),
 'host': 'UB20VM',
 'jobComplete': PosixPath('/home/paul/ePS/jobs/completed'),
 'jobPath': PosixPath('/home/paul/ePS/jobs'),
 'repoScpPath': PosixPath('/home/paul/python/epsman/repo'),
 'scpdir': PosixPath('/home/paul/python/epsman/shell'),
 'webDir': PosixPath('/home/paul/github/ePSdata'),
 'webScpPath': PosixPath('/home/paul/python/epsman/web'),
 'wrkdir': PosixPath('/home/paul/ePS')}

***Checking bin dirs
ePolyScat bin OK
***Anaconda bin not found


In [28]:
# Change from default ePS version to new build

# NOT YET WORKING
# job.setHostDefns({'ePSpath':'/opt/ePolyScat.3885d87/bin/ub-ifort/ePolyScat'})

# Manual
job.hostDefn['UB20VM']['ePSpath']=Path('/opt/ePolyScat.3885d87/bin/ub-ifort/ePolyScat')
# job.hostDefn['UB20VM']['scpdir']=Path('~/github/epsman/epsman/shell')
job.hostDefn['UB20VM']['scpdir']=Path('~/ePS/epsman/shell')

In [25]:
job.hostDefn['localhost']

{'host': 'jake',
 'IP': '127.0.0.1',
 'home': PosixPath('/home/paul'),
 'wrkdir': PosixPath('/home/paul/ePS/NO_2021/epsman_test_notebooks'),
 'webDir': PosixPath('/home/paul/github/ePSdata'),
 'localSettings': PosixPath('/home/paul/python/epsman/localSettings'),
 'systemDir': PosixPath('/home/paul/ePS/NO_2021/epsman_test_notebooks/NO_2021'),
 'elecDir': PosixPath('/home/paul/ePS/NO_2021/epsman_test_notebooks/NO_2021/electronic_structure'),
 'genDir': PosixPath('/home/paul/ePS/NO_2021/epsman_test_notebooks/NO_2021/generators'),
 'genFile': PosixPath('/home/paul/ePS/NO_2021/epsman_test_notebooks/NO_2021/generators/NO_symTest.orb1_S-0.conf'),
 'jobRoot': PosixPath('/home/paul/ePS/NO_2021/epsman_test_notebooks/NO_2021/NO_symTest'),
 'jobDir': PosixPath('/home/paul/ePS/NO_2021/epsman_test_notebooks/NO_2021/NO_symTest/orb1_S'),
 'webSystemDir': PosixPath('/home/paul/github/ePSdata/source/NO_2021')}

In [26]:
job.hostDefn['UB20VM']

{'host': 'UB20VM',
 'IP': '192.168.86.82',
 'home': PosixPath('/home/paul'),
 'wrkdir': PosixPath('/home/paul/ePS'),
 'scpdir': PosixPath('~/github/epsman/epsman/shell'),
 'jobPath': PosixPath('/home/paul/ePS/jobs'),
 'jobComplete': PosixPath('/home/paul/ePS/jobs/completed'),
 'ePSpath': PosixPath('/opt/ePolyScat.3885d87/bin/ub-ifort/ePolyScat'),
 'condaPath': PosixPath('/home/paul/anaconda3/bin/activate'),
 'condaEnv': 'base',
 'repoScpPath': PosixPath('/home/paul/python/epsman/repo'),
 'webScpPath': PosixPath('/home/paul/python/epsman/web'),
 'webDir': PosixPath('/home/paul/github/ePSdata'),
 'systemDir': PosixPath('/home/paul/ePS/NO_2021'),
 'elecDir': PosixPath('/home/paul/ePS/NO_2021/electronic_structure'),
 'genDir': PosixPath('/home/paul/ePS/NO_2021/generators'),
 'genFile': PosixPath('/home/paul/ePS/NO_2021/generators/NO_symTest.orb1_S-0.conf'),
 'jobRoot': PosixPath('/home/paul/ePS/NO_2021/NO_symTest'),
 'jobDir': PosixPath('/home/paul/ePS/NO_2021/NO_symTest/orb1_S'),
 'webSys

## Job configuration
Write file defining job. Save locally, transfer to remote machine.

In [12]:
# Set global job parameters + ePS molecule-specific inputs.
job.mol = 'NO_2021'  # Molecule name, used as root job dir + filename ID
job.orb = 'orb1_S'  # Orbital name, used as job dir + in filename
job.batch = 'NO_symTest'  # Set a name for the batch of jobs here, or as genFile below.
job.jobNote = 'NO sym test, orb 1 ionization (O 1s, S/A1).'  # Additional notes, included at inp file head.
job.elecStructure = 'NO_NIST_E_cc-pVDZ_ROHF.molden'  # Set just filename here, assumed to be on default paths?

# Changed... now set this to allow for jobRoot/orb dir format.
# Use Path(job.genFile.stem).stem here?  Or _ and split at _ TBD
job.setGenFile()  # Set genFile (configuration) name - used to define job below.
                    # Defaults {job.batch}.{job.orb}.conf if nothing passed.

Generator file set: NO_symTest.orb1_S.conf

*** Job paths set in self.hostDefn['UB20VM']:

{'IP': '192.168.86.82',
 'condaEnv': 'base',
 'condaPath': PosixPath('/home/paul/anaconda3/bin/activate'),
 'ePSpath': '/opt/ePolyScat.3885d87/bin/ub-ifort/ePolyScat',
 'elecDir': PosixPath('/home/paul/ePS/NO_2021/electronic_structure'),
 'genDir': PosixPath('/home/paul/ePS/NO_2021/generators'),
 'genFile': PosixPath('/home/paul/ePS/NO_2021/generators/NO_symTest.orb1_S.conf'),
 'home': PosixPath('/home/paul'),
 'host': 'UB20VM',
 'jobComplete': PosixPath('/home/paul/ePS/jobs/completed'),
 'jobDir': PosixPath('/home/paul/ePS/NO_2021/NO_symTest/orb1_S'),
 'jobPath': PosixPath('/home/paul/ePS/jobs'),
 'jobRoot': PosixPath('/home/paul/ePS/NO_2021/NO_symTest'),
 'repoScpPath': PosixPath('/home/paul/python/epsman/repo'),
 'scpdir': PosixPath('/home/paul/python/epsman/shell'),
 'systemDir': PosixPath('/home/paul/ePS/NO_2021'),
 'webDir': PosixPath('/home/paul/github/ePSdata'),
 'webScpPath': PosixPath('

In [13]:
job.hostDefn[job.host]

{'host': 'UB20VM',
 'IP': '192.168.86.82',
 'home': PosixPath('/home/paul'),
 'wrkdir': PosixPath('/home/paul/ePS'),
 'scpdir': PosixPath('/home/paul/python/epsman/shell'),
 'jobPath': PosixPath('/home/paul/ePS/jobs'),
 'jobComplete': PosixPath('/home/paul/ePS/jobs/completed'),
 'ePSpath': '/opt/ePolyScat.3885d87/bin/ub-ifort/ePolyScat',
 'condaPath': PosixPath('/home/paul/anaconda3/bin/activate'),
 'condaEnv': 'base',
 'repoScpPath': PosixPath('/home/paul/python/epsman/repo'),
 'webScpPath': PosixPath('/home/paul/python/epsman/web'),
 'webDir': PosixPath('/home/paul/github/ePSdata'),
 'systemDir': PosixPath('/home/paul/ePS/NO_2021'),
 'elecDir': PosixPath('/home/paul/ePS/NO_2021/electronic_structure'),
 'genDir': PosixPath('/home/paul/ePS/NO_2021/generators'),
 'genFile': PosixPath('/home/paul/ePS/NO_2021/generators/NO_symTest.orb1_S.conf'),
 'jobRoot': PosixPath('/home/paul/ePS/NO_2021/NO_symTest'),
 'jobDir': PosixPath('/home/paul/ePS/NO_2021/NO_symTest/orb1_S'),
 'webSystemDir': Po

In [14]:
# Testing for elec structure file path - may need as_posix here for win/lin client/host case? Not sure if it'll be an issue elsewhere?
print((job.hostDefn[job.host]['elecDir']/job.elecStructure).as_posix())

/home/paul/ePS/NO_2021/electronic_structure/NO_NIST_E_cc-pVDZ_ROHF.molden


### Version with looping over symmetries & spins for testing

In [ ]:
# Symmetry pairs for ScatSym (==ion x electron symm) and ScatContSym (==electron symm), input file will loop through these
# v1 - switched!
# Ssym=(P P P F F S A2 D)
# Csym=(S A2 D D G P P F)
# v2 16/12/20
# Ssym=(S A2 D D G P P P F)     #(S A2 D D G P P P F)
# Csym=(P P P F F S A2 D D)     #(P P P F F S A2 D D)


In [38]:
# Permutations of lists, from https://stackoverflow.com/questions/12935194/permutations-between-two-lists
# This is NOT what is required in this case.
# NOTE list2 must be equal or shorter than list1 here.

import itertools

# Set 1
# list2=['P', 'P', 'P', 'F', 'F', 'S', 'A2', 'D', 'D']   # Csym
# list1=['S', 'A2', 'D', 'D', 'G', 'P', 'P', 'P', 'F']  #Ssym

# Set 2
# list1=['P', 'P', 'P', 'F', 'F', 'S', 'A2', 'D', 'D']   # Ssym
# list2=['S', 'A2', 'D', 'D', 'G', 'P', 'P', 'P', 'F']  # Csym

# SpinDeg=4
# TargSym='P'		# Symmetry of the target state
# TargSpinDeg=3		# Target spin degeneracy
# InitSym='P'		# Initial state symmetry
# InitSpinDeg=2		# Initial state spin degeneracy'
# All failed


# # Set 3
# list2=['P', 'P', 'P', 'F', 'F', 'S', 'A2', 'D', 'D']   # Csym
# list1=['S', 'A2', 'D', 'D', 'G', 'P', 'P', 'P', 'F']  #Ssym

# # Global symmetries & spin
# # SpinDeg=2			# Spin degeneracy of the total scattering state (=1 singlet)
# SpinDeg=4
# TargSym='S'		# Symmetry of the target state
# TargSpinDeg=3		# Target spin degeneracy
# InitSym='P'		# Initial state symmetry
# InitSpinDeg=2		# Initial state spin degeneracy'


# *** ABOVE BASED ON DIRECT PRODUCTS, below just programatic (big computer, small brain appraoch)

# Set 4 - TRY ALL SYMMETRY COMBINATIONS!!!
# list2=['S', 'A2', 'B1', 'B2', 'P', 'D', 'F', 'G']   # Csym
# list1=['S', 'A2', 'B1', 'B2', 'P', 'D', 'F', 'G']  #Ssym

# Case for forced C2v symmetry
list1=['A1', 'A2', 'B1', 'B2']
list2=['A1', 'A2', 'B1', 'B2']

# TESTED FOR TargSym S and P
# # Global symmetries & spin
# # SpinDeg=2			# Spin degeneracy of the total scattering state (=1 singlet)
# SpinDeg=4
# TargSym='P'		# Symmetry of the target state
# TargSpinDeg=3		# Target spin degeneracy
# InitSym='P'		# Initial state symmetry
# InitSpinDeg=2		# Initial state spin degeneracy'

# out = [list(zip(x,list2)) for x in itertools.permutations(list1,len(list2))]

In [39]:
# This gives all permutations of complete lists
# list(itertools.permutations(list1,len(list2)))

# Product is good - gives all pairs from both lists
symList = list(itertools.product(list1, list2))

In [40]:
symList

[('A1', 'A1'),
 ('A1', 'A2'),
 ('A1', 'B1'),
 ('A1', 'B2'),
 ('A2', 'A1'),
 ('A2', 'A2'),
 ('A2', 'B1'),
 ('A2', 'B2'),
 ('B1', 'A1'),
 ('B1', 'A2'),
 ('B1', 'B1'),
 ('B1', 'B2'),
 ('B2', 'A1'),
 ('B2', 'A2'),
 ('B2', 'B1'),
 ('B2', 'B2')]

In [ ]:
# job.genFile

In [46]:
# Version with looping over symmetries
# nSym = 0
# This is now approximately working, although a fudge
# TODO: wrap as multi-class style job? Or just make a method for this

for nSym in range(0, len(symList)+1):  # [0]: # Quick test with ePS v3885d87 - [0] case throws errors?

    # Define job
    # For batch-chunked jobs, use
    #   job={Path(job.genFile.stem).stem}
    # or
    #   job=job.batch
    # To give consistent mol/batch/orb output tree.

    # This works as a basic templating engine.
    # f-string allows for replacement of var names (python >= 3.6)
    job.jobSettings = f"""

# Job definitions, used for dir structure and output naming.
mol={job.mol}
orb={job.orb}
job={job.batch}  
note='{job.jobNote}'


#*******************************************************************************************
# (c) Molecule (job) settings
#

# Electronic structure source
# elecStructure=$wrkdir/$mol/electronic_structure/NO_geom_cc-pVDZ.molden
elecStructure={(job.hostDefn[job.host]['elecDir']/job.elecStructure).as_posix()}

# IP (used also for FegeEng correction)
IP=9.26 # 1st vert IP, from NIST gas phase data https://webbook.nist.gov/cgi/cbook.cgi?ID=C10102439&Mask=20

# Additional commands when trying to get azulene working
# ECenter='0.0, 0.0, 0.0'

# Initial and final states by orbital occupation - 31 doubly occupied orbitals, here occupancy by degenracy.
# OrbOccInit='2 2 2 2 2 4 1'
# OrbOccTarget='1 2 2 2 2 4 1'
OrbOccInit='2 2 2 2 2 2 2 1 0'
OrbOccTarget='1 2 2 2 2 2 2 1 0'
# OrbOccInit='2 2 2 2 2 2 2 1'
# OrbOccTarget='1 2 2 2 2 2 2 1'

# CnvOrbSel='CnvOrbSel -9, (1, 9, 1, 9)'
# CnvOrbSel='CnvOrbSel -2, (19, 25, i=2, 9)'
# CnvOrbSel='CnvOrbSel 1, 30'	# Use CnvOrbSel in cases where additional orbitals are required (set to blank for defaults)
CnvOrbSel='CnvOrbSel 1, 9'
# CnvOrbSel=''

# Symmetry pairs for ScatSym (==ion x electron symm) and ScatContSym (==electron symm), input file will loop through these
# v1 - switched!
# Ssym=(P P P F F S A2 D)
# Csym=(S A2 D D G P P F)
# v2 16/12/20
# Ssym=(S A2 D D G P P P F)     #(S A2 D D G P P P F)
# Csym=(P P P F F S A2 D D)     #(P P P F F S A2 D D)
# With full symList loop
Ssym=({symList[nSym][0]})     
Csym=({symList[nSym][1]})
# Single sym test
# Ssym=(S)
# Csym=(P)

# Global symmetries & spin
SpinDeg=2			# Spin degeneracy of the total scattering state (=1 singlet)
# SpinDeg=4
TargSym='B1'		# Symmetry of the target state
TargSpinDeg=3		# Target spin degeneracy
InitSym='B1'		# Initial state symmetry
InitSpinDeg=2		# Initial state spin degeneracy'



    """

    # Write file locally (working dir).
    # with open(genFile + '.conf','w') as f:
    #     f.write(molSettings)
    #     print('Written local job conf file: ' + genFile + '.conf')
    # .... NOW handled in job.setGenFile()

    # Set generator file name (to be written)
    # Set to default to mol_orb.conf if not supplied.
    # Write file if jobSettings is configured.
    newFile = f'{job.batch}.{job.orb}-{nSym}.conf' # For nSym case, reset name to avoid overwrite
    # job.setGenFile(genFile=newFile)  # This currently doesn't work, since self.genFile is used in preference - set manually instead
#     y  # This works for n=0, then messes up? Possibly not pushing updated file?  Use energy labels to define below instead. NO - this just rewrites from same .conf file!!!
#     job.setGenFile()

    # Manual method: set filename, propagate with setGenFile, push file BEFORE job generation
    # TODO: fix some path inconsistencies here - in previous code self.genFile in wrkdir, and pushes OK, but should make consistent and explicit
    job.genFile = Path(newFile)
    job.setGenFile()
    job.writeGenFile()
    
    job.pushFile(job.genFile, job.hostDefn[job.host]['genDir'], overwritePrompt=False)  # OK for file in wrkdir, force overwrite - currently not working!!!
    
#     job.pushFile(job.genFile, job.hostDefn[job.host]['genDir'], overwritePrompt=False)  # OK for file in wrkdir, force overwrite - currently not working!!!
#     job.pushFile(Path(job.hostDefn['localhost']['wrkdir'], job.genDir), job.hostDefn[job.host]['genFile'])  # OK
#     job.pushFile(job.hostDefn['localhost']['genFile'], job.hostDefn[job.host]['genFile'])  # Need full path here of course!

#     job.Elist = em.multiEChunck(Estart=1.0, Estop=2.0, dE=1.0, EJob =2)  # Set for same range as previously, higher res.
    job.Elist = em.multiEChunck(Estart=nSym, Estop=nSym+1.0, dE=1.0, EJob =2)  # For sym test loop set as nSym to keep files distinct

    job.writeInp(scrType = 'basic', wLog = False)  #  'basic', 'wf-sph')

Generator file set: NO_symTest.orb1_S-0.conf

*** Job paths set in self.hostDefn['UB20VM']:

{'IP': '192.168.86.82',
 'condaEnv': 'base',
 'condaPath': PosixPath('/home/paul/anaconda3/bin/activate'),
 'ePSpath': PosixPath('/opt/ePolyScat.3885d87/bin/ub-ifort/ePolyScat'),
 'elecDir': PosixPath('/home/paul/ePS/NO_2021/electronic_structure'),
 'genDir': PosixPath('/home/paul/ePS/NO_2021/generators'),
 'genFile': PosixPath('/home/paul/ePS/NO_2021/generators/NO_symTest.orb1_S-0.conf'),
 'home': PosixPath('/home/paul'),
 'host': 'UB20VM',
 'jobComplete': PosixPath('/home/paul/ePS/jobs/completed'),
 'jobDir': PosixPath('/home/paul/ePS/NO_2021/NO_symTest/orb1_S'),
 'jobPath': PosixPath('/home/paul/ePS/jobs'),
 'jobRoot': PosixPath('/home/paul/ePS/NO_2021/NO_symTest'),
 'logFile': PosixPath('/home/paul/ePS/NO_2021/NO_symTest/orb1_S/NO_symTest.orb1_S-15.conf.log'),
 'repoScpPath': PosixPath('/home/paul/python/epsman/repo'),
 'scpdir': PosixPath('~/ePS/epsman/shell'),
 'systemDir': PosixPath('/ho

/home/paul/ePS/epsman/shell/ePS_input_write_template_basic.sh: line 37: /home/paul/ePS_stuff/scripts2019/machine.conf: No such file or directory


%%%%%%%%
Base directory: /home/paul/ePS/NO_2021/NO_symTest
Job dir: /home/paul/ePS/NO_2021/NO_symTest/orb1_S
MatE dir: /home/paul/ePS/NO_2021/NO_symTest/orb1_S/idy


mkdir: cannot create directory ‘/home/paul/ePS/NO_2021/NO_symTest’: File exists
mkdir: cannot create directory ‘/home/paul/ePS/NO_2021/NO_symTest/orb1_S’: File exists
mkdir: cannot create directory ‘/home/paul/ePS/NO_2021/NO_symTest/orb1_S/idy’: File exists


%%%%%%%%
Writing 2 energies to job file:
NO_symTest.orb1_S_E0.0_1.0_1.0eV
%%%%%%%%
Generator file set: NO_symTest.orb1_S-1.conf

*** Job paths set in self.hostDefn['UB20VM']:

{'IP': '192.168.86.82',
 'condaEnv': 'base',
 'condaPath': PosixPath('/home/paul/anaconda3/bin/activate'),
 'ePSpath': PosixPath('/opt/ePolyScat.3885d87/bin/ub-ifort/ePolyScat'),
 'elecDir': PosixPath('/home/paul/ePS/NO_2021/electronic_structure'),
 'genDir': PosixPath('/home/paul/ePS/NO_2021/generators'),
 'genFile': PosixPath('/home/paul/ePS/NO_2021/generators/NO_symTest.orb1_S-1.conf'),
 'home': PosixPath('/home/paul'),
 'host': 'UB20VM',
 'jobComplete': PosixPath('/home/paul/ePS/jobs/completed'),
 'jobDir': PosixPath('/home/paul/ePS/NO_2021/NO_symTest/orb1_S'),
 'jobPath': PosixPath('/home/paul/ePS/jobs'),
 'jobRoot': PosixPath('/home/paul/ePS/NO_2021/NO_symTest'),
 'logFile': PosixPath('/home/paul/ePS/NO_2021/NO_symTest/orb1_S/NO_symTest.orb1_S-0.conf.log'),
 'repoScpPath': PosixPath('/home/paul/python/epsma

...

/home/paul/ePS/epsman/shell/ePS_input_write_template_basic.sh: line 37: /home/paul/ePS_stuff/scripts2019/machine.conf: No such file or directory
mkdir: cannot create directory ‘/home/paul/ePS/NO_2021/NO_symTest’: File exists
mkdir: cannot create directory ‘/home/paul/ePS/NO_2021/NO_symTest/orb1_S’: File exists
mkdir: cannot create directory ‘/home/paul/ePS/NO_2021/NO_symTest/orb1_S/idy’: File exists


%%%%%%%%
Job: NO_symTest
File: NO_symTest.orb1_S_E14.0_1.0_15.0eV
Orb: orb1_S
Note: NO sym test, orb 1 ionization (O 1s, S/A1).
%%%%%%%%
Base directory: /home/paul/ePS/NO_2021/NO_symTest
Job dir: /home/paul/ePS/NO_2021/NO_symTest/orb1_S
MatE dir: /home/paul/ePS/NO_2021/NO_symTest/orb1_S/idy
%%%%%%%%
Writing 2 energies to job file:
NO_symTest.orb1_S_E14.0_1.0_15.0eV
%%%%%%%%
Generator file set: NO_symTest.orb1_S-15.conf

*** Job paths set in self.hostDefn['UB20VM']:

{'IP': '192.168.86.82',
 'condaEnv': 'base',
 'condaPath': PosixPath('/home/paul/anaconda3/bin/activate'),
 'ePSpath': PosixPath('/opt/ePolyScat.3885d87/bin/ub-ifort/ePolyScat'),
 'elecDir': PosixPath('/home/paul/ePS/NO_2021/electronic_structure'),
 'genDir': PosixPath('/home/paul/ePS/NO_2021/generators'),
 'genFile': PosixPath('/home/paul/ePS/NO_2021/generators/NO_symTest.orb1_S-15.conf'),
 'home': PosixPath('/home/paul'),
 'host': 'UB20VM',
 'jobComplete': PosixPath('/home/paul/ePS/jobs/completed'),
 'jobDir': PosixPath('

/home/paul/ePS/epsman/shell/ePS_input_write_template_basic.sh: line 37: /home/paul/ePS_stuff/scripts2019/machine.conf: No such file or directory
mkdir: cannot create directory ‘/home/paul/ePS/NO_2021/NO_symTest’: File exists
mkdir: cannot create directory ‘/home/paul/ePS/NO_2021/NO_symTest/orb1_S’: File exists
mkdir: cannot create directory ‘/home/paul/ePS/NO_2021/NO_symTest/orb1_S/idy’: File exists


Uploaded 
/home/paul/ePS/NO_2021/epsman_test_notebooks/NO_symTest.orb1_S-15.conf
 to 
/home/paul/ePS/NO_2021/generators/NO_symTest.orb1_S-15.conf
E = 15.0:1.0:16.0, 2 points total, 2/2 = 1 job files will be written.
Writing input files on remote...

%%%%%%%%
Job: NO_symTest
File: NO_symTest.orb1_S_E15.0_1.0_16.0eV
Orb: orb1_S
Note: NO sym test, orb 1 ionization (O 1s, S/A1).
%%%%%%%%
Base directory: /home/paul/ePS/NO_2021/NO_symTest
Job dir: /home/paul/ePS/NO_2021/NO_symTest/orb1_S
MatE dir: /home/paul/ePS/NO_2021/NO_symTest/orb1_S/idy
%%%%%%%%
Writing 2 energies to job file:
NO_symTest.orb1_S_E15.0_1.0_16.0eV
%%%%%%%%


IndexError: list index out of range

In [ ]:
# RESULTS 19/03/21 with above settings... close... but dropping at ScatStab?
# Looks like either convergence, renorm or threshold issue...?

:: WARNING: setvars.sh has already been run. Skipping re-execution.
   To force a re-execution of setvars.sh, use the '--force' option.
   Using '--force' can result in excessive use of your environment variables.
Processing NO_symTest.orb1_S_E0.0_1.0_1.0eV.inp file...
Processing NO_symTest.orb1_S_E10.0_1.0_11.0eV.inp file...
Processing NO_symTest.orb1_S_E1.0_1.0_2.0eV.inp file...
Processing NO_symTest.orb1_S_E11.0_1.0_12.0eV.inp file...
Processing NO_symTest.orb1_S_E12.0_1.0_13.0eV.inp file...
Processing NO_symTest.orb1_S_E13.0_1.0_14.0eV.inp file...
Processing NO_symTest.orb1_S_E14.0_1.0_15.0eV.inp file...
Processing NO_symTest.orb1_S_E15.0_1.0_16.0eV.inp file...
Processing NO_symTest.orb1_S_E2.0_1.0_3.0eV.inp file...
--------------------------------------------------------------------------
MPI_ABORT was invoked on rank 1 in communicator MPI_COMM_WORLD
with errorcode 1.

NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes.
You may or may not see output from other processes, depending on
exactly when Open MPI kills them.
--------------------------------------------------------------------------
[paul-VirtualBox:21080] 3 more processes have sent help message help-mpi-api.txt / mpi-abort
[paul-VirtualBox:21080] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages
Processing NO_symTest.orb1_S_E3.0_1.0_4.0eV.inp file...
Processing NO_symTest.orb1_S_E4.0_1.0_5.0eV.inp file...
Processing NO_symTest.orb1_S_E5.0_1.0_6.0eV.inp file...
Processing NO_symTest.orb1_S_E6.0_1.0_7.0eV.inp file...
Processing NO_symTest.orb1_S_E7.0_1.0_8.0eV.inp file...
--------------------------------------------------------------------------
MPI_ABORT was invoked on rank 2 in communicator MPI_COMM_WORLD
with errorcode 1.

NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes.
You may or may not see output from other processes, depending on
exactly when Open MPI kills them.
--------------------------------------------------------------------------
[paul-VirtualBox:21205] 3 more processes have sent help message help-mpi-api.txt / mpi-abort
[paul-VirtualBox:21205] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages
Processing NO_symTest.orb1_S_E8.0_1.0_9.0eV.inp file...
--------------------------------------------------------------------------
MPI_ABORT was invoked on rank 0 in communicator MPI_COMM_WORLD
with errorcode 1.

NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes.
You may or may not see output from other processes, depending on
exactly when Open MPI kills them.
--------------------------------------------------------------------------
[paul-VirtualBox:21230] 3 more processes have sent help message help-mpi-api.txt / mpi-abort
[paul-VirtualBox:21230] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages
Processing NO_symTest.orb1_S_E9.0_1.0_10.0eV.inp file...
paul@paul-VirtualBox:~/ePS/jobs$ 


In [ ]:
# SOLVED - issue was spin states
# ScatStab ends with C-GESV error
# Might be lib issue? 
# https://www.netlib.org/lapack/explore-html/d3/d16/group__complex_g_esolve_gaa9e9aeee4df582bfee679de7defb8e68.html

Number of asymptotic regions =      66
Final point in integration =   0.22782183E+03 Angstroms
Last asymptotic region is special region for dipole potential
Time Now =        21.6303  Delta time =        20.8654 End SolveHomo
myid is    1  info (=         1)  .ne. 0 - call to C-GESV (=         0)
myid is    1  ABSTOP message is VALUE SATISFIED CONDITION IN ABTSTI
     I WILL NOW TAKE THE LOG OF MINUS 1.0 TO CAUSE THE PROGRAM TO ABORT
                     NaN
myid is    2  info (=         1)  .ne. 0 - call to C-GESV (=         0)
myid is    2  ABSTOP message is VALUE SATISFIED CONDITION IN ABTSTI
     I WILL NOW TAKE THE LOG OF MINUS 1.0 TO CAUSE THE PROGRAM TO ABORT
                     NaN
myid is    3  info (=         1)  .ne. 0 - call to C-GESV (=         0)
myid is    3  ABSTOP message is VALUE SATISFIED CONDITION IN ABTSTI
     I WILL NOW TAKE THE LOG OF MINUS 1.0 TO CAUSE THE PROGRAM TO ABORT
                     NaN
Bad info in call to C-GESV    1
N        1  NRHS        1
     REAL PART - A on input
     ROW  1
  0.00000000E+00
     REAL PART - B on input
     ROW  1
  0.00000000E+00
     REAL PART - A after call to GESV
     ROW  1
  0.00000000E+00
IPIV pivot vector
    1
myid is    0  info (=         1)  .ne. 0 - call to C-GESV (=         0)
myid is    0  ABSTOP message is VALUE SATISFIED CONDITION IN ABTSTI
     I WILL NOW TAKE THE LOG OF MINUS 1.0 TO CAUSE THE PROGRAM TO ABORT
                     NaN


In [ ]:
# Check files only
# NOT WORKING FOR MULTIPLE SYM TESTING HERE.
# Run 
#   tail -f ls completed/Xe_symTest.orb19_4d_E*.inp.out
# on host as a quick check.

# FOR MULTIPLE SYM TEST need to fix genFile so that 
#    'ls ' + Path(self.hostDefn[self.host]['jobComplete'], self.genFile.stem 
# runs OK!
job.genFile = Path('NO_symTest.orb1_S.conf')
# job.genFile = Path('NO_symTest.orb1_S-0.conf')

# job.initConnection()
job.tidyJobs(chkFlag = True, mvFlag = False)

In [ ]:
# job.fAbrupt
# job.fileList

set(job.fileList) - set(job.fAbrupt)

In [ ]:
job.genFile

In [ ]:
# Set generator file name (to be written)
# Set to default to mol_orb.conf if not supplied.
# Write file if jobSettings is configured.
job.setGenFile()

In [ ]:
job.hostDefn['localhost']['genDir']

In [ ]:
job.hostDefn['localhost']  #['logFile']

In [ ]:
# Build remote dir tree, if necessary, and upload generator file
# em.createJobDirTree(wrkdir, mol, c, genFile = genFile + '.conf')
job.createJobDirTree();

In [ ]:
job.createJobDirTree(localHost=True)

**TODO** Check and copy/push electronic structure file here. See `_util.pushFile()` and `_epsJobGen.createJobDirTree()`

Also: win-lin file endings issue with generator file, see https://pythonconquerstheuniverse.wordpress.com/2011/05/08/newline-conversion-in-python-3/
Should also be able to do this with fabric, but didn't work it out yet.



In [ ]:
# File ending test...

# Existing code - defaults to local machine newline options (/n, /r, /n/r)
# with open(self.genFile,'w') as f:
#     f.write(self.jobSettings)

# THIS FIXES THE ISSUE - pusehd into main package routines
# with open(self.genFile,'w', newline="\n") as f:
#     f.write(self.jobSettings)


## Create multi-E job
Set Elist for job.

In [ ]:
job.Elist = em.multiEChunck(Estart=1.0, Estop=2.0, dE=1.0, EJob =2)  # Set for same range as previously, higher res.

## Write .inp files

In [ ]:
job.writeInp(scrType = 'basic')  #  'basic', 'wf-sph')

# Run jobs
Try this with 'nohup', but may need to run from terminal.

In [ ]:
# job.initConnection()
# job.runJobs()

#***** 
# nohup ./ePS_batch_job.sh &